In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [5]:
train_images[0].shape

(28, 28)

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16348\556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [9]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [10]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 09m 21s]
val_accuracy: 0.9129999876022339

Best val_accuracy So Far: 0.9138333201408386
Total elapsed time: 00h 42m 18s
INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        38448     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 64)                1486912   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,526,330
Trainable params: 1,526,330
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 152s 89ms/step - loss: 0.1412 - accuracy: 0.9479 - val_loss: 0.2554 - val_accuracy: 0.9153
Epoch 5/10
1688/1688 [==============================] - 128s 76ms/step - loss: 0.1017 - accuracy: 0.9625 - val_loss: 0.2728 - val_accuracy: 0.9097
Epoch 6/10
1688/1688 [==============================] - 129s 76ms/step - loss: 0.0737 - accuracy: 0.9730 - val_loss: 0.3293 - val_accuracy: 0.9053
Epoch 7/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.0518 - accuracy: 0.9818 - val_loss: 0.3408 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.0409 - accuracy: 0.9848 - val_loss: 0.3673 - val_accuracy: 0.9138
Epoch 9/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.0291 - accuracy: 0.9895 - val_loss: 0.4691 - val_accuracy: 0.9145
Epoch 10/10
1688/1688 [==============================] - 126s 75ms/step - loss: 0.0285 - accuracy: 0.9899 - val_loss: 